In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile

import pandas as pd
import numpy as np


     |████████████████████████████████| 51kB 3.1MB/s 
     |████████████████████████████████| 1.3MB 8.0MB/s 


In [2]:
# jobName = '201216_Train_pilot'
jobName = '201216_Train_split1'
x, y = LibData.GetXY_s_o(jobName)

# ov 컬럼 생성
ovName = 'pv_ov'
y[ovName] = y.apply(lambda line: line['pv_fee'] - line['pv_claim'], axis=1)

rf = LibML.RandomForestDefault(len(x.columns))
xgb = LibML.XGBoostDefault()

In [3]:
# 각각
print(LibML.GetScoresHoldOut(x,y,rf,0.2), LibML.GetScoresHoldOut(x,y,xgb,0.2))

[161.11, 0.1935, 0.9133] [163.06, 0.1961, 0.9318]


In [4]:
# 한번에
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

x, x_, y, y_ = train_test_split(x, y, test_size=0.2)

rf.fit(x,y[ovName])
pred_ov = pd.Series(rf.predict(x_), index=x_.index)
scoresRF = LibUtils.GetScores(y_[ovName], pred_ov)

xgb.fit(x,y[ovName])
pred_ov = pd.Series(xgb.predict(x_), index=x_.index)
scoresXGB = LibUtils.GetScores(y_[ovName], pred_ov)

print(scoresRF, scoresXGB)

[189.75, 0.2317, 0.8864] [181.15, 0.2212, 0.9306]


In [ ]:
from xgboost import XGBRegressor

# fee
model = XGBRegressor(max_delta_step=0.0, objective = 'reg:squarederror', nthread=-1,
                     n_estimators = 5000, colsample_bytree=0.9168,learning_rate=0.1284,max_depth=20,min_child_weight=5,subsample=0.4541)
preds_f = LibML.GetPredCV(l_x, l_y, model, 'pv_fee')
preds_f = LibML.GetMinZero(preds_f)
maes_f = LibML.GetMaesCV(l_y, preds_f, 'pv_fee')

# claim
preds_c = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
preds_c = LibML.GetMinZero(preds_c)
maes_c = LibML.GetMaesCV(l_y, preds_c, 'pv_claim')

# get ov
preds_derived_ov = LibML.GetDeduct(preds_f, preds_c)
maes_derived_ov = LibML.GetMaesCV(l_y, preds_derived_ov, 'pv_ov')

# ov
preds_ov = LibML.GetPredCV(l_x, l_y, model, 'pv_ov')
maes_ov = LibML.GetMaesCV(l_y, preds_ov, 'pv_ov')

tag1 = 'each vs ov, xgboost'
LibUtils.WriteNote(tag1, 'fee', [str(sum(maes_f)/5), maes_f])
LibUtils.WriteNote(tag1, 'claim', [str(sum(maes_c)/5), maes_c])
LibUtils.WriteNote(tag1, 'derived_ov', [str(sum(maes_derived_ov)/5), maes_derived_ov])
LibUtils.WriteNote(tag1, 'ov', [str(sum(maes_ov)/5), maes_ov])

LibUtils.ELoop()

In [ ]:
from xgboost import XGBRegressor
# ov
model = XGBRegressor(max_delta_step=0.0, objective = 'reg:squarederror', nthread=-1,
                     n_estimators = 5000, colsample_bytree=0.9168,learning_rate=0.1284,max_depth=20,min_child_weight=5,subsample=0.4541)

preds_ov = LibML.GetPredCV(l_x, l_y, model, 'pv_ov')
maes_ov = LibML.GetMaesCV(l_y, preds_ov, 'pv_ov')

tag1 = 'each vs ov, xgboost'
LibUtils.WriteNote(tag1, 'ov', [str(sum(maes_ov)/5), maes_ov])

LibUtils.ELoop()


start learning 1 2020-12-26 04:25... end learning 2020-12-26 05:20
start learning 2 2020-12-26 05:20... end learning 2020-12-26 06:13
start learning 3 2020-12-26 06:13... end learning 2020-12-26 07:07
start learning 4 2020-12-26 07:07... end learning 2020-12-26 08:01
start learning 5 2020-12-26 08:01... end learning 2020-12-26 08:54
start empty loop.............................................................

KeyboardInterrupt: ignored